## Lab 4: Unsupervised Learning


Dataset: https://www.kaggle.com/datasets/uciml/breast-cancer-wisconsin-data


Steps:
1. PCA implementation
2. K-means implementation

Then:

3. Exp1: Cluster using k-means + apply elbow method
4. Exp2: Cluster using PCA then k-means + apply elbow method
5. Apply Elbow Method for both experiments
6. Calculate errors for both experiments
7. Try different numbers of PC


8. OG labels VS output
9. Validate via PCA sklearn
